In [99]:
import json
from datetime import datetime,timedelta
import calendar

In [100]:
# Opening JSON file
f = open('trainings.txt')
data = json.load(f)
f.close()

In [101]:
# Function to get the most recent completion for each training
def get_most_recent_completions(completions):
    most_recent = {}
    for training in completions:
        date = datetime.strptime(training["timestamp"], "%m/%d/%Y")
        if training["name"] not in most_recent or date > most_recent[training["name"]]["timestamp"]:
            most_recent[training["name"]] = {"timestamp": date, "expires": training.get("expires")}
    return most_recent

In [102]:
# Task 1
# List each completed training with a count of how many people have completed that training.
complted_count = {}
for people in data:
    recent_completions = get_most_recent_completions(people["completions"])
    for trainings in recent_completions:
        complted_count[trainings] = complted_count.get(trainings, 0) + 1
            
cn_to_json = json.dumps(complted_count, indent=4)
 
with open("completed_training_count.json", "w") as outfile:
    outfile.write(cn_to_json)

In [103]:
# Task 2
# Listing all people that completed that training in the specified fiscal year (2024).
# With Trainings = "Electrical Safety for Labs", "X-Ray Safety", "Laboratory Safety Training
def list_people(training_list,fiscal_year):
    training_completed_list = {training: [] for training in training_list}

    start_date = datetime(fiscal_year-1, 7, 1)
    end_date = datetime(fiscal_year, 6, 30)
    
    for people in data:
        recent_completions = get_most_recent_completions(people["completions"])
        for trainings in recent_completions:
            if trainings in training_list and start_date <= recent_completions[trainings]["timestamp"] <= end_date:
                training_completed_list[trainings].append(people["name"])
                   
    cn_to_json = json.dumps(training_completed_list, indent=4)

    with open("training_list.json", "w") as outfile:
        outfile.write(cn_to_json)
        

training_list = ["Electrical Safety for Labs", "X-Ray Safety", "Laboratory Safety Training"]
fiscal_year = 2024
list_people(training_list,fiscal_year)

In [104]:
# Task 3
# Find all people that have any completed trainings that have already expired, or will expire within one month
from operator import itemgetter
def find_trainingexpired_people(givendate):
    givendate_obj = datetime.strptime(givendate, "%m/%d/%Y")
    training_expiry_list = {}
    for people in data:
        recent_completions = get_most_recent_completions(people["completions"])
        for trainings in recent_completions:
            if recent_completions[trainings]["expires"] is not None:
                expiresdate_obj = datetime.strptime(recent_completions[trainings]["expires"], "%m/%d/%Y")
                days_in_month = calendar.monthrange(givendate_obj.year, givendate_obj.month)[1]
               
                if givendate_obj >= expiresdate_obj:
                    training_expiry_list.setdefault(people["name"], []).append({"training_name":trainings,"expires":"Expired"})
                elif (givendate_obj + timedelta(days=days_in_month)) >= expiresdate_obj: # next one month which expires soon 
                    training_expiry_list.setdefault(people["name"], []).append({"training_name":trainings,"expires":"expires soon"})
                   
    cn_to_json = json.dumps(training_expiry_list, indent=4)

    with open("training_expiry_list.json", "w") as outfile:
        outfile.write(cn_to_json)          
    
# Given data Oct 1st, 2023
givendate = "10/1/2023"
find_trainingexpired_people(givendate)